In [1]:
import imaplib
import smtplib
import email
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.header import decode_header
import time
from textblob import TextBlob
# Gmail credentials
IMAP_SERVER = 'imap.gmail.com'
SMTP_SERVER = 'smtp.gmail.com'
EMAIL = 'amababhale@gmail.com'
PASSWORD = 'pyxv wzbo vlou jmai'  # Use an App Password
# Categorized response templates
RESPONSE_TEMPLATES = {
    "greeting": (
        "Hello,\n\n"
        "Thank you for your email. It is always a pleasure to hear from you. Please let us know how we can assist you today. Whether you have a specific inquiry or need general information, our team is here to help.\n\n"
        "We look forward to your response.\n\n"
        "Best regards,\nBabhale Infotech"
   ),
    "query": (
        "Hello,\n\n"
        "Thank you for reaching out with your question. Your inquiry is important to us, and we are committed to providing you with a detailed and comprehensive response.\n\n"
        "Please allow us some time to review your query thoroughly. We aim to respond with all the information you need as quickly as possible. If there are any updates or additional details you would like to share in the meantime, please don’t hesitate to let us know.\n\n"
        "Best regards,\nBabhale Infotech"
    ),
    "complaint": (
        "Hello,\n\n"
        "We sincerely apologize for any inconvenience or dissatisfaction you have experienced. Your feedback is invaluable to us, and we are committed to resolving this issue promptly and effectively.\n\n"
        "Our team has been notified and is already looking into the matter. We will keep you informed throughout the resolution process. If there is anything further you would like to share or any additional details you believe might help, please don’t hesitate to let us know.\n\n"
        "Thank you for bringing this to our attention.\n\n"
        "Best regards,\nBabhale Infotech"
    ),
    "follow_up": (
        "Hello,\n\n"
        "Thank you for following up on your previous email. We greatly appreciate your patience as we work to address your concern.\n\n"
        "Our team is actively reviewing your case, and we are committed to providing you with an update as soon as possible. Your satisfaction is our priority, and we will ensure your inquiry is handled with care and urgency.\n\n"
        "Thank you for your understanding and continued support.\n\n"
        "Best regards,\nBabhale Infotech"
    ),
    "feedback": (
        "Hello,\n\n"
        "Thank you for taking the time to provide your feedback. Your insights are incredibly valuable to us as we strive to improve our services and exceed your expectations.\n\n"
        "We will carefully review your suggestions and work to incorporate them into our future plans. If you have any additional thoughts or ideas, we would be delighted to hear them.\n\n"
        "Thank you for helping us grow and improve.\n\n"
        "Best regards,\nBabhale Infotech"
    ),
    "default": (
        "Hello,\n\n"
        "Thank you for your email. We appreciate your message and are committed to responding as soon as possible. Our team is currently reviewing your inquiry, and we will ensure you receive a comprehensive response at the earliest opportunity.\n\n"
        "In the meantime, if there is anything additional you would like to share, please feel free to reach out again.\n\n"
        "Best regards,\nBabhale Infotech"
    ),
    "promotion": (
        "Hello,\n\n"
        "Thank you for sharing your promotional offer with us. We value partnerships and are always open to exploring new opportunities that align with our goals and vision.\n\n"
        "Our team will review the details of your proposal carefully and get back to you if there is potential for collaboration. If you have any additional information or materials to share, please don’t hesitate to forward them.\n\n"
        "Best regards,\nBabhale Infotech"
    ),
    "thank_you": (
        "Hello,\n\n"
        "We are truly grateful for your kind words and support. Your appreciation motivates us to continue delivering our best and striving for excellence.\n\n"
        "If there is ever anything further we can assist you with, please don’t hesitate to reach out. We look forward to serving you again in the future.\n\n"
        "Best regards,\nBabhale Infotech"
    ),
}

def fetch_unread_emails():
    """Fetch unread emails from the Gmail inbox."""
    try:
        mail = imaplib.IMAP4_SSL(IMAP_SERVER)
        mail.login(EMAIL, PASSWORD)
        mail.select("inbox")

        status, messages = mail.search(None, 'UNSEEN')
        email_ids = messages[0].split()
        print(f"Found {len(email_ids)} unread emails.")
        return mail, email_ids
    except Exception as e:
        print(f"Error fetching emails: {e}")
        return None, []

def analyze_email_content(subject, body):
    """Analyze the email content to determine the category."""
    text = f"{subject} {body}"
    analysis = TextBlob(text)
  # Example logic for categorization
    if any(word in text.lower() for word in ["hello", "hi", "greetings"]):
        return "greeting"
    elif any(word in text.lower() for word in ["query", "question", "inquire"]):
        return "query"
    elif any(word in text.lower() for word in ["complaint", "issue", "problem"]):
        return "complaint"
    elif any(word in text.lower() for word in ["follow up", "checking", "reminder"]):
        return "follow_up"
    elif any(word in text.lower() for word in ["feedback", "suggestion"]):
        return "feedback"
    elif any(word in text.lower() for word in ["promotion", "offer", "discount"]):
        return "promotion"
    elif any(word in text.lower() for word in ["thank you", "thanks"]):
        return "thank_you"
    else:
        return "default"

def send_auto_reply(to_email, subject, category):
    """Send an automatic reply to the sender based on the category."""
    try:
        smtp_server = smtplib.SMTP(SMTP_SERVER, 587)
        smtp_server.starttls()
        smtp_server.login(EMAIL, PASSWORD)

        msg = MIMEMultipart()
        msg['From'] = EMAIL
        msg['To'] = to_email
        msg['Subject'] = f"Re: {subject}"

        body = RESPONSE_TEMPLATES.get(category, RESPONSE_TEMPLATES["default"])
        msg.attach(MIMEText(body, 'plain'))

        smtp_server.sendmail(EMAIL, to_email, msg.as_string())
        print(f"Auto-reply sent to: {to_email} | Category: {category}")
        smtp_server.quit()
    except Exception as e:
        print(f"Error sending reply to {to_email}: {e}")

def main():
    """Main function to fetch unread emails and send auto-replies."""
    while True:
        mail, email_ids = fetch_unread_emails()
        if email_ids:
            for email_id in email_ids:
                status, data = mail.fetch(email_id, '(RFC822)')
                for response_part in data:
                    if isinstance(response_part, tuple):
                        msg = email.message_from_bytes(response_part[1])
                        subject = decode_header(msg["Subject"])[0][0]
                        subject = subject.decode() if isinstance(subject, bytes) else subject
                        from_email = msg.get("From").split("<")[-1].strip(">")
                       # Extract email body
                        if msg.is_multipart():
                            for part in msg.walk():
                                if part.get_content_type() == "text/plain":
                                    body = part.get_payload(decode=True).decode()
                                    break
                        else:
                            body = msg.get_payload(decode=True).decode()

                        print(f"Processing email from: {from_email} | Subject: {subject}")

                        category = analyze_email_content(subject, body)
                        send_auto_reply(from_email, subject, category)
        else:
            print("No unread emails found.")

        # Pause before checking for new emails
        time.sleep(5)  # Adjust the sleep time as needed

#if _name_ == "_main_":
 #   main()
if __name__ == "__main__":
    main()


Found 2 unread emails.
Processing email from: aditya.katle2003@gmail.com | Subject: 
Auto-reply sent to: aditya.katle2003@gmail.com | Category: default
Processing email from: workspace-noreply@google.com | Subject: Annual Reminder about Google Terms and Policies
Auto-reply sent to: workspace-noreply@google.com | Category: greeting
Found 0 unread emails.
No unread emails found.


KeyboardInterrupt: 